<a href="https://colab.research.google.com/github/vyavasthita/dsml_learning/blob/master/pandas/pfizer_case_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown 173A59xh2mnpmljCCB9bhC4C5eP2IS6qZ

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('Pfizer_1.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

# Melting
Convert columns to rows.


Structure the data in a more readable form.

In [ ]:
pd.melt(data, id_vars=['Date', 'Drug_Name', 'Parameter']).head(20)

Use proper name for generated columns variable and value

In [ ]:
data_melt = pd.melt(data, id_vars=['Date', 'Drug_Name', 'Parameter'], var_name='time', value_name='reading')
data_melt

# Pivoting
### Opposite of Melting

In [ ]:
data_melt.pivot(index=['Date', 'Drug_Name', 'Parameter'], columns='time', values='reading')

In [ ]:
data_melt.pivot(index=['Date', 'Drug_Name', 'Parameter'], columns='time', values='reading').reset_index()


 We can see the column parameter has two values, pressure and temperature and they are getting repeated.

We can convert these two values to two columns using Pivoting.

In [ ]:
data_melt

In [ ]:
data_tidy = data_melt.pivot(index=['time', 'Date', 'Drug_Name'], columns='Parameter', values='reading')
data_tidy

In [ ]:
data_tidy.reset_index(inplace=True)
data_tidy

Categorising temperature values into different readable texts.

So we need to convert numerical data to different bins.

In [ ]:
# first find out the min and max values of Temperature

In [ ]:
data_tidy['Temperature'].min()

In [ ]:
data_tidy['Temperature'].max()

In [ ]:
temp_points = [5, 20, 35, 50, 60]
temp_labels = ['low', 'medium', 'high', 'very high']
# (5, 20] exclude 5 and include 20 --> low
# (20, 35] starting 21 and ending 34 --> medium

In [ ]:
data_tidy['temp_cat'] = pd.cut(data_tidy['Temperature'], bins=temp_points, labels=temp_labels)
data_tidy

In [ ]:
data_tidy['temp_cat'].value_counts()

# Missing Values
NaN vs None
Nan => Not a Number, used to represent numerical columns (int and float)
None => Used to represent missing values in non-numerical columns (string)

If we use None in a numerical column, it will automatically get converted to NaN

In [ ]:
pd.Series([1, np.nan, 3, None])

In [ ]:
data.isna() # Count missing values

In [ ]:
data.isnull() # just an alias for isna

In [ ]:
# Count the missing values
# We can not use value_counts() because we need to use it for all columns
data.isna().sum() # sum of column values (axis = 0 by default)

In [ ]:
# Row wise missing count
data.isna().sum(axis = 1) # sum of row values

So far, we have found out the missing values, now we need to decide what to do with the missing values.
Dealing with missing values.

In [ ]:
# Option 1 (if na count is very very low, we can simply drop them as the remaining data is large enough and deleting a few will not impact the result.)
data.dropna() # remove row if any columns contains na

In [ ]:
data.dropna(axis=1) # drop columns having na

But we will lose lots of data here if we drop because our dataset is very, very small with just 18 rows. So, drop will not be the right choice here.

We need to use **data imputation** => mean, mode, median, 0, custom values etc.

String Methods

In [ ]:
data_tidy['Drug_Name']

In [ ]:
# Requirement: Convert Drug_Name to Upper case
# we can not directly apply upper() because data_tidy['Drug_Name'] is a Series object.
# We need to convert this to string first
data_tidy['Drug_Name'].str.upper()

In [ ]:
# Find hydrochloride
data_tidy['Drug_Name'].str.contains('hydrochloride')

In [ ]:
data_tidy[data_tidy['Drug_Name'].str.contains('hydrochloride')]

In [ ]:
# Split date, month and year in data_tidy['Date]
data_tidy['Date'].str.split('-')

In [ ]:
def extract_year(date):
  return date[2]

In [ ]:
data_tidy['Date'].str.split('-').apply(lambda date: date[2]).astype('int')

In [ ]:
# We can do better using datetimes
# Combine date and time
data_tidy['Date'] + ' ' + data_tidy['time']

In [ ]:
# Here both Date and time columns are of string type and we are doing string concatenation,
# so the output column timestamp is also of string type
data_tidy['timestamp'] = data_tidy['Date'] + ' ' + data_tidy['time']
data_tidy

Convert column to datetime for easy calculation on date and time objects

In [ ]:
data_tidy['timestamp'] = pd.to_datetime(data_tidy['timestamp'])
data_tidy

In [ ]:
data_tidy.info() # timestamp of datetime type

In [ ]:
data_tidy['timestamp'].dt.hour
data_tidy['timestamp'].dt.month
data_tidy['timestamp'].dt.year

In [ ]:
data_tidy['timestamp'].dt.month_name()